# The first part of the assignment - Use beautifualsoup to scrpy Tornato neighborhood data from wikipedia.

#### import package

In [129]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

#### The wikipeida URL:https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

In [2]:
url = r'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

#### use request and BeautifulSoup to get postcal data from the URL

In [4]:
html = requests.get(url)

In [5]:
soup = BeautifulSoup(html.text)

#### check tags in html and transform html information to text

In [6]:
soup.find('table').find_all('td')[4].get_text()

'\nM5ADowntown Toronto(Regent Park / Harbourfront)\n\n'

#### use for loop to create the postcal list

In [7]:
output_list = []
for cell_text in soup.find('table').find_all('td'):
    tmp_postcode = cell_text.find('b').get_text()
    a_text = cell_text.find_all('a')
    if len(a_text) > 1:
        tmp_borough = a_text[0].get_text()
        tmp_neighbor = []
        for neighbor_idx in range(1, len(a_text)):
            tmp_neighbor.append(a_text[neighbor_idx].get_text())
        tmp_neighbor = ', '.join(tmp_neighbor)
        output_list.append([tmp_postcode, tmp_borough, tmp_neighbor])
    elif len(a_text) == 1:
        tmp_borough = a_text[0].get_text()
        output_list.append([tmp_postcode, tmp_borough, tmp_borough])

#### transform the postcal list to dataframe

In [8]:
output_table = pd.DataFrame(output_list, columns = ['PostalCode', 'Borough', 'Neighborhood'])
output_table.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Queen's Park


In [9]:
output_table.shape

(101, 3)

# The Second part of the assignment - Use csv to add coordinates on postcal code table

#### Due to that the geocode package lost connection. Thus, read the coordination data from local.

In [130]:
geo_data = pd.read_csv('Geospatial_Coordinates.csv')
geo_data.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


#### Merge the coordination data to the postal code data, whiche achive in the first part of the assignment.

In [131]:
output_table_new = pd.merge(output_table, geo_data, left_on ='PostalCode', right_on = 'Postal Code')
output_table_new.head()

,PostalCode,Borough,Neighborhood,Postal Code,Latitude,Longitude
0,M3A,North York,Parkwoods,M3A,43.753259,-79.329656
1,M4A,North York,Victoria Village,M4A,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",M5A,43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",M6A,43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,M7A,43.662301,-79.389494


#### Because the postal code columns are duplicate, I drop one of them.

In [12]:
output_table_new.drop(columns = 'Postal Code', inplace = True)
output_table_new.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494


In [13]:
output_table_new.shape

(101, 5)

# # The Third part of the assignment - Use the similar method to analyze Tornato

In [84]:
from geopy.geocoders import Nominatim
import folium 
from pandas.io.json import json_normalize
import numpy as np
from sklearn.cluster import KMeans
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# 3-1 Prepare Neighborhoods data in Tornato

#### Change the variable name of the dataframe achieved from the second part of the assignment.

In [18]:
neighborhoods = output_table_new.drop(columns = 'PostalCode')

In [19]:
tornato_data = neighborhoods[neighborhoods['Borough'].str.contains('Toronto')]

In [40]:
tornato_data = tornato_data.reset_index(drop = True)
tornato_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
2,Downtown Toronto,St. James Town,43.651494,-79.375418
3,Downtown Toronto,Downtown Toronto,43.644771,-79.373306
4,Downtown Toronto,Bay Street,43.657952,-79.387383


#### Use geopy library to get the latitude and longitude values of Tornato.

In [132]:
address = 'Downtown Toronto, Canada'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6563221, -79.3809161.


#### Create a map of Tornato with neighborhoods superimposed on top.

In [137]:
# create map of Tornato using latitude and longitude values
map_tornato = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(tornato_data['Latitude'], tornato_data['Longitude'], tornato_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_tornato)  
    
map_tornato

#### Define Foursquare Credentials and Version


In [34]:
CLIENT_ID = 'EUYBJPFTP1O4XS41AXXNWGCCVDVTGGNL0AB4Z3L0NBUYSMA2' # your Foursquare ID
CLIENT_SECRET = 'PW0F1XTMO5ZP3PLZE4C3LIKLD4C40XYJ1BCZSZPFFERZGK3F' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: EUYBJPFTP1O4XS41AXXNWGCCVDVTGGNL0AB4Z3L0NBUYSMA2
CLIENT_SECRET:PW0F1XTMO5ZP3PLZE4C3LIKLD4C40XYJ1BCZSZPFFERZGK3F


# 3-2 Explore Neighborhoods in Tornato

#### The same funciton creates to repeat the same process to all the neighborhoods in Tornato

In [138]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [109]:
tornato_venues = getNearbyVenues(tornato_data.Neighborhood, tornato_data.Latitude, tornato_data.Longitude)

Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
Downtown Toronto
Bay Street
Downtown Toronto
Richmond, King
Harbourfront, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
North Toronto
University of Toronto
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront, South Niagara, Island airport
Rosedale
Downtown Toronto
St. James Town, Cabbagetown
First Canadian Place, Underground city
Church and Wellesley


#### Let's check the size of the resulting dataframe

In [110]:
print(tornato_venues.shape)
tornato_venues.head()

(1216, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,"Regent Park, Harbourfront",43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant


In [111]:
tornato_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Bay Street,63,63,63,63,63,63
"CN Tower, King and Spadina, Railway Lands, Harbourfront, South Niagara, Island airport",17,17,17,17,17,17
Church and Wellesley,77,77,77,77,77,77
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
Downtown Toronto,176,176,176,176,176,176
"First Canadian Place, Underground city",100,100,100,100,100,100
"Garden District, Ryerson",100,100,100,100,100,100
"Harbourfront, Union Station, Toronto Islands",100,100,100,100,100,100
"Kensington Market, Chinatown, Grange Park",59,59,59,59,59,59


#### Let's find out how many unique categories can be curated from all the returned venues

In [112]:
print('There are {} uniques categories.'.format(len(tornato_venues['Venue Category'].unique())))

There are 206 uniques categories.


## 3-3. Analyze Each Neighborhood

In [113]:
# one hot encoding
tornato_onehot = pd.get_dummies(tornato_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
tornato_onehot['Neighborhood'] = tornato_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [tornato_onehot.columns[-1]] + list(tornato_onehot.columns[:-1])
tornato_onehot = tornato_onehot[fixed_columns]

tornato_onehot.head()

,Yoga Studio,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


And let's examine the new dataframe size.

In [114]:
tornato_onehot.shape

(1216, 206)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [115]:
tornato_grouped = tornato_onehot.groupby('Neighborhood').mean().reset_index()
tornato_grouped

,Neighborhood,Yoga Studio,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar
0,Bay Street,0.015873,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.015873,0.000000,0.000000,0.00,0.00,0.015873,0.00000,0.000000,0.015873
1,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.000000,0.000000,0.058824,0.058824,0.058824,0.117647,0.117647,0.117647,...,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.00000,0.000000,0.000000
2,Church and Wellesley,0.025974,0.012987,0.012987,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.012987,0.012987,0.012987,0.00,0.00,0.000000,0.00000,0.000000,0.000000
3,"Commerce Court, Victoria Hotel",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.010000,0.020000,0.000000,0.000000,0.00,0.00,0.020000,0.00000,0.000000,0.010000
4,Downtown Toronto,0.005682,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.011364,0.000000,0.000000,0.00,0.00,0.011364,0.00000,0.000000,0.000000
5,"First Canadian Place, Underground city",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.010000,0.020000,0.010000,0.000000,0.00,0.01,0.010000,0.00000,0.000000,0.010000
6,"Garden District, Ryerson",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.010000,0.010000,0.020000,0.000000,0.00,0.00,0.000000,0.00000,0.010000,0.010000
7,"Harbourfront, Union Station, Toronto Islands",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.010000,0.000000,0.010000,0.000000,0.00,0.01,0.010000,0.00000,0.000000,0.010000
8,"Kensington Market, Chinatown, Grange Park",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.050847,0.00000,0.067797,0.016949
9,North Toronto,0.047619,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.00000,0.000000,0.000000


#### Let's confirm the new size

In [116]:
tornato_grouped.shape

(17, 206)

#### Let's print each neighborhood along with the top 5 most common venues

In [117]:
num_top_venues = 5

for hood in tornato_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = tornato_grouped[tornato_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Bay Street----
                venue  freq
0         Coffee Shop  0.19
1                Café  0.06
2  Italian Restaurant  0.05
3      Sandwich Place  0.05
4         Salad Place  0.03


----CN Tower, King and Spadina, Railway Lands, Harbourfront, South Niagara, Island airport----
              venue  freq
0  Airport Terminal  0.12
1    Airport Lounge  0.12
2   Airport Service  0.12
3             Plane  0.06
4       Coffee Shop  0.06


----Church and Wellesley----
                 venue  freq
0          Coffee Shop  0.06
1  Japanese Restaurant  0.06
2     Sushi Restaurant  0.06
3              Gay Bar  0.04
4           Restaurant  0.04


----Commerce Court, Victoria Hotel----
         venue  freq
0  Coffee Shop  0.13
1   Restaurant  0.07
2         Café  0.06
3        Hotel  0.06
4          Gym  0.04


----Downtown Toronto----
           venue  freq
0    Coffee Shop  0.11
1   Cocktail Bar  0.05
2           Café  0.04
3       Beer Bar  0.03
4  Grocery Store  0.03


----First Canadian Pl

#### Let's put that into a _pandas_ dataframe
First, let's write a function to sort the venues in descending order.

In [118]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [119]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = tornato_grouped['Neighborhood']

for ind in np.arange(tornato_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(tornato_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bay Street,Coffee Shop,Café,Italian Restaurant,Sandwich Place,Salad Place,Japanese Restaurant,Bubble Tea Shop,Pizza Place,Burger Joint,Diner
1,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Terminal,Airport Lounge,Airport Service,Plane,Coffee Shop,Rental Car Location,Boat or Ferry,Boutique,Bar,Airport Gate
2,Church and Wellesley,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Gay Bar,Restaurant,Yoga Studio,Dance Studio,Fast Food Restaurant,Pub,Men's Store
3,"Commerce Court, Victoria Hotel",Coffee Shop,Restaurant,Café,Hotel,Gym,Italian Restaurant,Seafood Restaurant,Japanese Restaurant,American Restaurant,Cocktail Bar
4,Downtown Toronto,Coffee Shop,Cocktail Bar,Café,Bakery,Restaurant,Grocery Store,Seafood Restaurant,Beer Bar,Park,Pharmacy


<a id='item4'></a>


## 3-4. Cluster Neighborhoods


Run _k_-means to cluster the neighborhood into 5 clusters.

In [120]:
# set number of clusters
kclusters = 5

tornato_grouped_clustering = tornato_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(tornato_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 1, 0, 0, 0, 0, 0, 0, 4, 3], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [121]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
neighborhoods_venues_sorted.head()

,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,0,Bay Street,Coffee Shop,Café,Italian Restaurant,Sandwich Place,Salad Place,Japanese Restaurant,Bubble Tea Shop,Pizza Place,Burger Joint,Diner
1,1,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Terminal,Airport Lounge,Airport Service,Plane,Coffee Shop,Rental Car Location,Boat or Ferry,Boutique,Bar,Airport Gate
2,0,Church and Wellesley,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Gay Bar,Restaurant,Yoga Studio,Dance Studio,Fast Food Restaurant,Pub,Men's Store
3,0,"Commerce Court, Victoria Hotel",Coffee Shop,Restaurant,Café,Hotel,Gym,Italian Restaurant,Seafood Restaurant,Japanese Restaurant,American Restaurant,Cocktail Bar
4,0,Downtown Toronto,Coffee Shop,Cocktail Bar,Café,Bakery,Restaurant,Grocery Store,Seafood Restaurant,Beer Bar,Park,Pharmacy


In [122]:
tornato_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
2,Downtown Toronto,St. James Town,43.651494,-79.375418
3,Downtown Toronto,Downtown Toronto,43.644771,-79.373306
4,Downtown Toronto,Bay Street,43.657952,-79.387383


In [123]:
tornato_merged = pd.merge(tornato_data,neighborhoods_venues_sorted, on ='Neighborhood')
tornato_merged.head()

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,0,Coffee Shop,Park,Bakery,Breakfast Spot,Theater,Pub,Café,Event Space,French Restaurant,Mexican Restaurant
1,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,0,Coffee Shop,Clothing Store,Italian Restaurant,Hotel,Japanese Restaurant,Bubble Tea Shop,Cosmetics Shop,Café,Middle Eastern Restaurant,Fast Food Restaurant
2,Downtown Toronto,St. James Town,43.651494,-79.375418,0,Coffee Shop,Café,Cosmetics Shop,Cocktail Bar,Gastropub,Park,Department Store,Farmers Market,Restaurant,Clothing Store
3,Downtown Toronto,Downtown Toronto,43.644771,-79.373306,0,Coffee Shop,Cocktail Bar,Café,Bakery,Restaurant,Grocery Store,Seafood Restaurant,Beer Bar,Park,Pharmacy
4,Downtown Toronto,Downtown Toronto,43.669542,-79.422564,0,Coffee Shop,Cocktail Bar,Café,Bakery,Restaurant,Grocery Store,Seafood Restaurant,Beer Bar,Park,Pharmacy


Finally, let's visualize the resulting clusters

In [125]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(tornato_merged['Latitude'], tornato_merged['Longitude'], tornato_merged['Neighborhood'], tornato_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

#### The above map show the cluster result of neighborhoods in differnt colors.